In [1]:
import edward as ed
import tensorflow as tf
from edward.models import Normal, Binomial, Multinomial
import numpy as np

/Users/daniekbrink/opt/anaconda3/envs/testenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/daniekbrink/opt/anaconda3/envs/testenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/daniekbrink/opt/anaconda3/envs/testenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/daniek

In [2]:
def generator(arrays, batch_size):
    """Generate batches, one with respect to each array's first axis."""
    starts = [0] * len(arrays)  # pointers to where we are in iteration
    while True:
        batches = []
        for i, array in enumerate(arrays):
            start = starts[i]
            stop = start + batch_size
            diff = stop - array.shape[0]
            if diff <= 0:
                batch = array[start:stop]
                starts[i] += batch_size
            else:
                batch = np.concatenate((array[start:], array[:diff]))
                starts[i] = diff
            batches.append(batch)
        yield batches


In [3]:
N = 10
max_berries = 20
X = tf.placeholder(tf.float32, [1])
mu_w = Normal(loc=tf.ones(1)*0.02, scale=tf.ones(1))
sig_w = Normal(loc=tf.ones(1)*0.5, scale=tf.ones(1))

Y = Multinomial(total_count=tf.ones(1)*X, probs=[0.3, 0.7])
W = Normal(loc=mu_w, scale=sig_w)
Xt = Normal(loc=X - Y[:,0] + W, scale=tf.ones(1)*5)

yw = Multinomial(total_count=tf.get_variable("Y/total_count", [1]),
               probs=tf.get_variable("Y/probs", [1, 2]))

muw = Normal(loc=tf.get_variable("mu_w/loc", [1]),
            scale=tf.nn.softplus(tf.get_variable("mu_w/scale", [1])))
sigw = Normal(loc=tf.get_variable("sig_w/loc", [1]),
             scale=tf.nn.softplus(tf.get_variable("sig_w/scale", [1])))

X_train = [12, 10, 8, 6, 5, 4, 3, 3, 2, 1]
y_train = [10, 8, 6, 5, 4, 3, 3, 2, 1, 0]

M = 1
data = generator([np.asarray(X_train), np.asarray(y_train)], M)

n_batch = int(N / M)
n_epoch = 5
inference = ed.KLqp({mu_w: muw, sig_w:sigw}, data={})
inference.initialize(
    n_iter=n_batch * n_epoch, n_samples=5, scale={Xt: N / M})
tf.global_variables_initializer().run()

for _ in range(inference.n_iter):
    X_batch, y_batch = next(data)
    info_dict = inference.update({X: X_batch, Xt: y_batch})
    inference.print_progress(info_dict)

x_post = ed.copy(Xt, {mu_w: muw, sig_w:sigw})
sess=ed.get_session()
for i in range(0,N):
    print(sess.run(x_post.mean(), feed_dict={X:[X_train[i]]}))

[11.032847]
[8.527079]
[9.784733]
[5.20264]
[1.9581648]
[0.7855418]
[1.7039595]
[2.6208446]
[2.3175178]
[1.9690039]


In [4]:
X_batch

array([1])

In [5]:
y_batch

array([0])